The below code creates the hamiltonian for the following TSP formulation (taken from qiskit)
$C(\textbf{x})=\sum_{i,j=1}^{N-1}w_{ij}\sum_{p=1}^{N-1} x_{i,p}x_{j,p+1}+\sum_{j=1}^{N-1}w_{0j} x_{j,1}+\sum_{i=1}^{N-1}w_{i0} x_{i,N-1}+ A\sum_{p=1}^{N-1}\left(1- \sum_{i=1}^{N-1} x_{i,p}\right)^2+A\sum_{i=1}^{N-1}\left(1- \sum_{p=1}^{N-1} x_{i,p}\right)^2.$



In [2]:
from pyquil.paulis import PauliTerm, PauliSum

# city 0 is fixed as the first and last city visited
def get_qubit(city, order, num_cities):
    return (city-1) * (num_cities-1) + order

# assumes cities are labelled from 0 to N-1
# uses Qiskit formulation: $$C(\\textbf{x})=\\sum_{i,j=1}^{N-1}w_{ij}\\sum_{p=1}^{N-1} x_{i,p}x_{j,p+1}+\\sum_{j=1}^{N-1}w_{0j} x_{j,1}+\\sum_{i=1}^{N-1}w_{i0} x_{i,N-1}+ A\\sum_{p=1}^{N-1}\\left(1- \\sum_{i=1}^{N-1} x_{i,p}\\right)^2+A\\sum_{i=1}^{N-1}\\left(1- \\sum_{p=1}^{N-1} x_{i,p}\\right)^2.$$
def tsp_hamiltonian(graph, A):
    N = len(graph.nodes())
    num_qubits = N ** 2
    hamiltonian = []

    for city in range(1, N):
        terms = []
        terms.append(PauliTerm("I", 0, 1))
        for i in range(1, N):
            qubit = get_qubit(city, i, N)
            terms.append(PauliTerm("Z", qubit, -1))

        hamiltonian.append(PauliTerm("I", 0, A) * PauliSum(terms) * PauliSum(terms))

    for i in range(1, N):
        terms = []
        terms.append(PauliTerm("I", 0, 1))
        for city in range(1, N):
            qubit = get_qubit(city, i, N)
            terms.append(PauliTerm("Z", qubit, -1))

        hamiltonian.append(PauliTerm("I", 0, A) * PauliSum(terms) * PauliSum(terms))



    for i in range(1, N):
        for j in range(1, N):
            w = graph[i,j]['weight']

            terms = []
            for p in range(1, N):
                qubit_i = get_qubit(i, p, N)
                qubit_j = get_qubit(j, p+1, N)
                terms.append(PauliTerm("Z", qubit_i, 1) * PauliTerm("Z", qubit_j, 1))

            hamiltonian.append(PauliTerm("I", 0, w) * PauliSum(terms))



    terms = []
    for j in range(1, N):
        w_0j = graph[0, j]['weight']
        qubit = get_qubit(j, 1, N)
        terms.append(PauliTerm("Z", qubit, w_0j))
    hamiltonian.append(PauliSum(terms))


    terms = []
    for i in range(1, N):
        w_i0 = graph[i, 0]['weight']
        qubit = get_qubit(i, N-1, N)
        terms.append(PauliTerm("Z", qubit, w_i0))
    hamiltonian.append(PauliSum(terms))
    
    return hamiltonian
